In [ ]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help, proxy_sim
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

# Main TEST

In [ ]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 5, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 5, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 5, 
    "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 5, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 3, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=3)
print(tensor_stimuli)


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



pd.set_option('display.max_rows', None)

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



# Specific Grid Test

In [ ]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)

# test stimuli
test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

# test grid  
grid_mu = torch.linspace(start = .2, end  = .8, steps = 5)
grid_sigma = torch.linspace(start = .2, end  = .8, steps = 5)
grid_y = torch.linspace(start = .2, end  = .8, steps = 5)
grid_epsilon = torch.linspace(start = .2, end  = .8, steps = 5)

# test prior
PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 3, "max_observation": 500
}


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

params = init_params_tensor.granch_params(
                grid_mu =  grid_mu,
                grid_sigma = grid_sigma,
                grid_y = grid_y, 
                grid_epsilon = grid_epsilon,
                hypothetical_obs_grid_n = 5, 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)

res

# Visualizing random grid KL

In [48]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


def run_random_grid_and_visualize_kl():

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    BATCH_INFO = {
        "jitter_n": 1, 
        "total_batch_n": 1, 
        "jitter_mode": "sampling"
    }

    GRID_INFO = {
        "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 5, 
        "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 5, 
        "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 5, 
        "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 5, 
        "hypothetical_obs_grid_n": 10
    }


    BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

    PRIOR_INFO = {
        "mu_prior": 1,  
        "V_prior": 1, 
        "alpha_prior": 3, 
        "beta_prior": 1, 
        "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
        "hypothetical_obs_grid_n": 2, 
        "world_EIGs": 3, "max_observation": 500
    }


    test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
    test_stimuli.add_toy_example(0.3, 0.7)



    tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

    params = init_params_tensor.granch_params(
                    grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                    grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                    grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                    grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                    hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                    mu_prior = PRIOR_INFO["mu_prior"],
                    V_prior = PRIOR_INFO["V_prior"], 
                    alpha_prior = PRIOR_INFO["alpha_prior"], 
                    beta_prior = PRIOR_INFO["beta_prior"],
                    epsilon  = PRIOR_INFO["epsilon"], 
                    mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                    sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                    world_EIGs = PRIOR_INFO["world_EIGs"],
                    max_observation = PRIOR_INFO["max_observation"], 
                    forced_exposure_max= np.nan, 
                    linking_hypothesis = "surprisal")
            
                # add the various different cached bits
    params.add_meshed_grid()
    params.add_lp_mu_sigma()
    params.add_y_given_mu_sigma()
    params.add_lp_epsilon()
    params.add_priors()

    if params.linking_hypothesis == "EIG": 
        res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
    else: 
        res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)

    return res


#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



# try it out 5 eims
df_list = []
for i in range(500): 
    print(i)
    res = run_random_grid_and_visualize_kl()
    res["id"] = i
    df_list.append(res)
main_res = pd.concat(df_list)
main_res.to_csv("visualize_kl_turbo.csv")
#main_res.dropna()
#print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



0
1


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30013808608055115' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29992952942848206' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3001295328140259' has dtype incompatible with int64, please explicitly cast to a compatible d

2
3
4
5


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30009809136390686' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999279797077179' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000538647174835' has dtype incompatible with int64, please explicitly cast to a compatible dt

6
7


...

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2998024821281433' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3001726269721985' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30017393827438354' has dtype incompatible with int64, please explicitly cast to a compatible dt

494
495
496


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000074625015259' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999729812145233' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999816834926605' has dtype incompatible with int64, please explicitly cast to a compatible dty

497
498
499


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2998100817203522' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30000436305999756' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000401556491852' has dtype incompatible with int64, please explicitly cast to a compatible dt

In [45]:
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

df = main_res

# Get unique categories
id = df['id'].unique()
# Create a subplot grid
fig = sp.make_subplots(rows=len(id), cols=1, subplot_titles=["1", "2", "3", "4", "5"])

# Populate each subplot
for i, idx in enumerate(id):
    subset = df[df['id'] == idx]
    print(subset)
    single_fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)
    single_fig.update_layout(
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
    )


    for trace in single_fig.data:
        fig.add_trace(trace, row=i + 1, col=1)

# Update subplot layout (optional)
fig.update_layout(
    title='Faceted Line Plots',
    showlegend=True,
    height=600,
    width=400,
)

# Show the plot
fig.show()


       t       kl1       kl2  id
0      0  0.016368  0.029003   0
1      1       0.0       0.0   0
2      2       0.0       0.0   0
3      3       0.0       0.0   0
4      4       0.0       0.0   0
5      5       0.0       0.0   0
6      6       0.0       0.0   0
7      7       0.0       0.0   0
8      8       0.0       0.0   0
9      9       0.0       0.0   0
10    10       0.0       0.0   0
11    11       0.0       0.0   0
12    12       0.0       0.0   0
13    13       0.0       0.0   0
14    14       0.0       0.0   0
15    15       0.0       0.0   0
16    16       0.0       0.0   0
17    17       0.0       0.0   0
18    18       0.0       0.0   0
19    19       0.0       0.0   0
20    20       0.0       0.0   0
21    21       0.0       0.0   0
22    22       0.0       0.0   0
23    23       0.0       0.0   0
24    24       0.0       0.0   0
25    25       0.0       0.0   0
26    26       0.0       0.0   0
27    27       0.0       0.0   0
28    28       0.0       0.0   0
29    29  

       t       kl1         kl2  id
0      0  0.016368    0.029003   0
1      1       0.0         0.0   0
2      2       0.0         0.0   0
3      3       0.0         0.0   0
4      4       0.0         0.0   0
5      5       0.0         0.0   0
6      6       0.0         0.0   0
7      7       0.0         0.0   0
8      8       0.0         0.0   0
9      9       0.0         0.0   0
10    10       0.0         0.0   0
11    11       0.0         0.0   0
12    12       0.0         0.0   0
13    13       0.0         0.0   0
14    14       0.0         0.0   0
15    15       0.0         0.0   0
16    16       0.0         0.0   0
17    17       0.0         0.0   0
18    18       0.0         0.0   0
19    19       0.0         0.0   0
20    20       0.0         0.0   0
21    21       0.0         0.0   0
22    22       0.0         0.0   0
23    23       0.0         0.0   0
24    24       0.0         0.0   0
25    25       0.0         0.0   0
26    26       0.0         0.0   0
27    27       0.0  

In [ ]:
import pandas as pd
import plotly.express as px



df = res.iloc[:50]

# Create a line plot using Plotly Express
fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)

# Customize the layout (optional)
fig.update_layout(
    title='Line Plot with Plotly',
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
)

# Show the plot
fig.show()


# Toy Example Test 

In [ ]:
importlib.reload(init_model_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(proxy_sim)

test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = 3,
                #world_EIGs = PRIOR_INFO["world_EIGs"],
                #max_observation = PRIOR_INFO["max_observation"], 
                max_observation = 500,
                forced_exposure_max= np.nan, 
                linking_hypothesis= "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

pd.set_option('display.max_rows', None)

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)
#res.behavior

res.behavior

In [ ]:
res.behavior